# Experiment - Loughran & McDonald Difference-in-difference

In [1]:
NAME = '09-04_experiment_lm_did'
PROJECT = 'conference-calls-sentiment'
PYTHON_VERSION = '3.7.0'

### Imports  

In [2]:
import os
import re
import numpy as np
import pandas as pd

# Regressions
from linearmodels import PanelOLS
import statsmodels.api as sm

# Hide annoying warnings
import warnings
warnings.filterwarnings('ignore')

### Settings

In [3]:
workdir = re.sub("(?<={})[\w\W]*".format(PROJECT), "", os.getcwd())
os.chdir(workdir)

pipeline = os.path.join('2_pipeline', NAME)
if not os.path.exists(pipeline):
    os.makedirs(pipeline)
    for folder in ['out', 'store', 'tmp']:
        os.makedirs(os.path.join(pipeline, folder))

--- 
# Main code

# Preprocess Transcripts

In [ ]:
controls = pd.read_feather(os.path.join('2_pipeline', '07-01_dataset_control_variables', 'out', 'control_variables.feather'))
controls.head()

In [ ]:
LM_PUBLICATION = '2011-01-06'  # Date of Loughran & McDonald (2011) Publication
tone_by_role = pd.read_feather(os.path.join('2_pipeline', '04-01_tone_lm', 'out', 'lm_tone_by_role.feather'))

tone_did = (tone_by_role
            .assign(year=lambda x: x['event_date'].dt.year,
                    quarter=lambda x: x['event_date'].dt.to_period('Q'),
                    treated=lambda x: np.where(x['speaker_role'] == 'Management', 1, 0),
                    post=lambda x: (x['event_date'] >= LM_PUBLICATION).astype('int'),
                    treated_post=lambda x: x['post'] * x['treated'])
            .merge(controls, on=['gvkey', 'quarter'])
            .assign(quarter=lambda x: x['quarter'].astype('str'))  # Stata cannot handle the pandas.Period dtype
            .dropna()
            .set_index(['gvkey', 'year']))

tone_did

In [32]:
tone_did['treated'].value_counts()

0    22293
1    22289
Name: treated, dtype: int64

In [33]:
# Save
tone_did.to_stata(os.path.join(pipeline, 'out', 'tone_did.dta'))

## Diffrence-in-difference

In [34]:
controls = ['surprise', 'size', 'roa', 'leverage', 'loss', 'capex']

In [35]:
def ols_model(df, dependent, exog, ffe, tfe):
    dependent = df[dependent]
    exog = sm.add_constant(df[exog])
    model = PanelOLS(dependent, exog, entity_effects=ffe, time_effects=tfe)
    return model.fit(cov_type='clustered', cluster_entity=True)

### Negative Words

In [37]:
ols_model(df=tone_did, 
          dependent='lm_negative',
          exog=['treated', 'post', 'treated_post' ,'num_words'],
          ffe=False,
          tfe=False)

Dep. Variable:,lm_negative,R-squared:,0.5858
Estimator:,PanelOLS,R-squared (Between):,0.6215
No. Observations:,44582,R-squared (Within):,0.5690
Date:,"Thu, May 13 2021",R-squared (Overall):,0.5858
Time:,20:08:42,Log-likelihood,-1.602e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.576e+04
Entities:,601,P-value,0.0000
Avg Obs:,74.180,Distribution:,"F(4,44577)"
Min Obs:,1.0000,,
Max Obs:,136.00,F-statistic (robust):,1049.2


In [11]:
ols_model(df=tone_did, 
          dependent='lm_negative',
          exog=['treated', 'post', 'treated_post', 'num_words'] + controls,
          ffe=False,
          tfe=False)

Dep. Variable:,lm_negative,R-squared:,0.5904
Estimator:,PanelOLS,R-squared (Between):,0.6374
No. Observations:,44582,R-squared (Within):,0.5713
Date:,"Thu, May 13 2021",R-squared (Overall):,0.5904
Time:,20:02:05,Log-likelihood,-1.599e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,6423.4
Entities:,601,P-value,0.0000
Avg Obs:,74.180,Distribution:,"F(10,44571)"
Min Obs:,1.0000,,
Max Obs:,136.00,F-statistic (robust):,454.43


In [12]:
ols_model(df=tone_did, 
          dependent='lm_negative',
          exog=['treated', 'post', 'treated_post', 'num_words'],
          ffe=False,
          tfe=True)

Dep. Variable:,lm_negative,R-squared:,0.5853
Estimator:,PanelOLS,R-squared (Between):,0.4737
No. Observations:,44582,R-squared (Within):,0.5174
Date:,"Thu, May 13 2021",R-squared (Overall):,0.5258
Time:,20:02:05,Log-likelihood,-1.596e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.572e+04
Entities:,601,P-value,0.0000
Avg Obs:,74.180,Distribution:,"F(4,44561)"
Min Obs:,1.0000,,
Max Obs:,136.00,F-statistic (robust):,1029.3


In [13]:
ols_model(df=tone_did, 
          dependent='lm_negative',
          exog=['treated', 'post', 'treated_post', 'num_words'],
          ffe=True,
          tfe=True)

Dep. Variable:,lm_negative,R-squared:,0.5703
Estimator:,PanelOLS,R-squared (Between):,0.4664
No. Observations:,44582,R-squared (Within):,0.5147
Date:,"Thu, May 13 2021",R-squared (Overall):,0.5226
Time:,20:02:06,Log-likelihood,-1.55e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.459e+04
Entities:,601,P-value,0.0000
Avg Obs:,74.180,Distribution:,"F(4,43961)"
Min Obs:,1.0000,,
Max Obs:,136.00,F-statistic (robust):,1439.6


In [14]:
ols_model(df=tone_did, 
          dependent='lm_positive',
          exog=['treated', 'post', 'treated_post', 'num_words'],
          ffe=False,
          tfe=False)

Dep. Variable:,lm_positive,R-squared:,0.7881
Estimator:,PanelOLS,R-squared (Between):,0.6291
No. Observations:,44582,R-squared (Within):,0.8077
Date:,"Thu, May 13 2021",R-squared (Overall):,0.7881
Time:,20:02:06,Log-likelihood,-1.759e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,4.145e+04
Entities:,601,P-value,0.0000
Avg Obs:,74.180,Distribution:,"F(4,44577)"
Min Obs:,1.0000,,
Max Obs:,136.00,F-statistic (robust):,1931.2


In [15]:
ols_model(df=tone_did, 
          dependent='lm_positive',
          exog=['treated', 'post', 'treated_post', 'num_words'] + controls,
          ffe=False,
          tfe=False)

Dep. Variable:,lm_positive,R-squared:,0.7903
Estimator:,PanelOLS,R-squared (Between):,0.6423
No. Observations:,44582,R-squared (Within):,0.8085
Date:,"Thu, May 13 2021",R-squared (Overall):,0.7903
Time:,20:02:07,Log-likelihood,-1.757e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.679e+04
Entities:,601,P-value,0.0000
Avg Obs:,74.180,Distribution:,"F(10,44571)"
Min Obs:,1.0000,,
Max Obs:,136.00,F-statistic (robust):,847.14


In [16]:
ols_model(df=tone_did, 
          dependent='lm_positive',
          exog=['treated', 'post', 'treated_post', 'num_words'],
          ffe=False,
          tfe=True)

Dep. Variable:,lm_positive,R-squared:,0.7882
Estimator:,PanelOLS,R-squared (Between):,0.2195
No. Observations:,44582,R-squared (Within):,0.7169
Date:,"Thu, May 13 2021",R-squared (Overall):,0.6836
Time:,20:02:07,Log-likelihood,-1.756e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,4.147e+04
Entities:,601,P-value,0.0000
Avg Obs:,74.180,Distribution:,"F(4,44561)"
Min Obs:,1.0000,,
Max Obs:,136.00,F-statistic (robust):,5063.4


In [17]:
ols_model(df=tone_did, 
          dependent='lm_positive',
          exog=['treated', 'post', 'treated_post', 'num_words'],
          ffe=True,
          tfe=True)

Dep. Variable:,lm_positive,R-squared:,0.8084
Estimator:,PanelOLS,R-squared (Between):,0.6017
No. Observations:,44582,R-squared (Within):,0.8054
Date:,"Thu, May 13 2021",R-squared (Overall):,0.7845
Time:,20:02:08,Log-likelihood,-1.701e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,4.638e+04
Entities:,601,P-value,0.0000
Avg Obs:,74.180,Distribution:,"F(4,43961)"
Min Obs:,1.0000,,
Max Obs:,136.00,F-statistic (robust):,1988.4
